# Models

## Task definition:
Our aim is to define model that will work with unbalanced data, what can we achieve by Deep Learning methods with the proper metrics and loss functions.

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

print('GPU: ', torch.backends.mps.is_available()) 
print('GPU built: ', torch.backends.mps.is_built())
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print('device: ', device)

GPU:  True
GPU built:  True
device:  mps


In [2]:
# function faster than pd.dummies
def transofrm(data):
    
    label_encoder = LabelEncoder()

    for column in data.columns:
        if data[column].dtype == 'object':  
            data[column] = label_encoder.fit_transform(data[column])

    return data

In [3]:
df_train = pd.read_csv('fraudTrain.csv')
df_test = pd.read_csv('fraudTest.csv')
df_train = df_train.drop(columns='Unnamed: 0')
df_test = df_test.drop(columns='Unnamed: 0')
df_train = df_train.dropna()
df_test = df_test.dropna()
df_train.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [4]:
df_train = transofrm(df_train)
train_features = df_train.drop(columns=['is_fraud']).values
print('features number: ', train_features.shape[1])
train_labels = df_train['is_fraud'].values

df_test= transofrm(df_test)
test_features = df_train.drop(columns=['is_fraud']).values
test_labels = df_train['is_fraud'].values

features number:  21


## Deep Learing Metrics

-Sensitivity and Specificity are better for imbalanced data

Especially crucial is sensitivity because it measures correctly classified positive samples with respect to the total number of positive samples

In [5]:
count_classes = df_train['is_fraud'].value_counts()

possitive_class = count_classes[1]
negative_class = count_classes[0]

Positive class is much fewer represented that's why the model supposed to pay more attention to it during training.

In [6]:
positive_class_weight = int(count_classes[0] / count_classes[1])
negative_class_weight = 1

print(positive_class_weight)
print(negative_class_weight)

criterion = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(positive_class_weight)).to(device) 
# pos_weight=torch.Tensor([positive_class_weight, negative_class_weight])

171
1


## ANN Model

In [7]:
from torch.nn import Linear
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_features, output_classes):
        super(MLP, self).__init__()
        torch.manual_seed(12345)
        self.lin1 = nn.Linear(input_features, input_features*2)
        self.lin2 = nn.Linear(input_features*2, input_features)
        self.lin3 = nn.Linear(input_features, input_features // 2)
        self.lin4 = nn.Linear(input_features // 2, output_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.relu(x)
        x = self.lin3(x)
        x = F.relu(x)
        x = self.lin4(x)
        
        return x

model = MLP(train_features.shape[1], 1).to(device)
print(model)

MLP(
  (lin1): Linear(in_features=21, out_features=42, bias=True)
  (lin2): Linear(in_features=42, out_features=21, bias=True)
  (lin3): Linear(in_features=21, out_features=10, bias=True)
  (lin4): Linear(in_features=10, out_features=1, bias=True)
)


### Dataset

In [8]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Training
During training we can see how not trustworthy accuracy is for unbalanced data, after one epoch (with Sensitivity equal to 0 and huge loss) we achieve over 99% accuracy. That's why it is important to be aware of distributions that the dataset follows and properties measured by our metrics. 
That example shows also how ineffective could be early on the test set results, even if they don't improve much, we see a huge improvement in the loss function.

In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.unsqueeze(1).to(device)  
        out = model(inputs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader.dataset)

def test(model, test_loader, device):
    model.eval()
    true_positives = 0
    actual_positives = 0
    total_samples = 0
    correct_predictions = 0
    
    for batch in test_loader:
        features, labels = batch
        features, labels = features.to(device), labels.to(device)  
        out = model(features)
        pred = out.argmax(dim=1)
        
        true_positives += (pred == 1).logical_and(labels == 1).sum().item()
        actual_positives += labels.sum().item()
        correct_predictions += (pred == labels).sum().item()
        total_samples += len(labels)

    sensitivity = true_positives / actual_positives if actual_positives > 0 else None
    accuracy = correct_predictions / total_samples if total_samples > 0 else None
    
    return sensitivity, accuracy

num_epochs = 30
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

for epoch in range(num_epochs):
    loss = train(train_loader, model, criterion, optimizer, device)

    sensitivity, test_acc = test(model, test_loader, device)
    print(f'Epoch: {epoch+1:02d}, Loss: {loss}, Test Sensitivity: {sensitivity}, Test Accuracy: {test_acc}')

Epoch: 001, Loss: 568264321588.1188, Test Sensitivity: 0.0, Test Accuracy: 0.9942113482561166
Epoch: 002, Loss: 229926043107.41568, Test Sensitivity: 0.0, Test Accuracy: 0.9942113482561166
Epoch: 003, Loss: 10021040342.686415, Test Sensitivity: 0.0, Test Accuracy: 0.9942113482561166
Epoch: 004, Loss: 0.02149232729539414, Test Sensitivity: 0.0, Test Accuracy: 0.9942113482561166
Epoch: 005, Loss: 0.021489179444464282, Test Sensitivity: 0.0, Test Accuracy: 0.9942113482561166
